## GBP for CisDecoding batch
### outputing heatmap and weight raw data

Visualize the CRE bins relevant to the prediction of target class.
Input: h5 trained model, CRE arrays (in npy format) used for training with "1dCNN CisDecoding_training_basic.py" located on the "select_GBP" directory
Output: heatmap and raw (numerical) data for Guided-Backpropagation on "GBPheatmap-cisdecode" and "GBPrawdata-cisdecode" directories, respectively.

To run, "helper_forCisDecode2.py" and "visualizations_forCisDecode.py" are required in the same directory.

In [ ]:
import gc
import matplotlib.pyplot as plt
import numpy as np
from visualizations_forCisDecode import GradCAM, GuidedGradCAM, GBP, LRP, CLRP, SGLRP, LRPA, LRPB, LRPE
from keras.layers import (Activation, Add, GlobalAveragePooling2D,
                          BatchNormalization, Conv1D, Conv2D, Dense, Flatten, Reshape, Input, Dropout,
                          MaxPooling1D,MaxPooling2D)

from keras.models import load_model, Model, Sequential
from helper_forCisDecode2 import heatmap_optional
import innvestigate.utils as iutils
import os
from keras.utils import plot_model,np_utils
from keras.preprocessing.image import img_to_array, load_img

from keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras import backend as K
from keras import optimizers

from functools import reduce
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from sklearn.metrics import classification_report, confusion_matrix

import skimage as sk
sk.__version__

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
input_shape=(82055,20,50,1)### adjust the shape used in the training. The current setting is for 20-bins with 50 TF channels
num_classes=2

from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    model = load_model("./example.h5")### load tarined model for prediction of binary expression pattern from CRE arrays

print(model.summary())

In [ ]:
### often need to run some times: 1-4 times ###
partial_model = Model(
    inputs=model.inputs,
    outputs=iutils.keras.graph.pre_softmax_tensors(model.outputs),
    name=model.name,
)
partial_model.summary()

In [ ]:
max_input = 1
min_input = 0

In [ ]:
### Guided-bakpropagation for selected CRE arrays in npy format, located on the "select_GBP" directory  ###
import glob

imagefolder = "./GBPheatmap-cisdecode/" ### heatmap folder
rawfolder = "./GBPrawdata-cisdecode/"   ### raw data folder

np.set_printoptions(precision=8, suppress=True, linewidth=1000)
guidedbackprop_analyzer = GBP(
    partial_model,
    target_id=1,### backprop for "positive" class: BRup for the original data
    relu=False,
)


### Heatmap setting ###
rate = 0.6 ### threshold value for visualization, in relative relevance level of 0.0-1.0
alpha = 1 ### alpha value
transp = "False" ### transparency in the background
# cmap = "seismic"

sfiles = glob.glob("./select_GBP/*.npy") ### /selected/内の.npyファイル名を取得
for sfile in sfiles:
    ### heatmap ###
    input_img = np.load(sfile)
    input_img_1 = np.reshape(input_img,(1,20,50))
    analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_img_1)
    fig = plt.figure(figsize=(20, 8), dpi=600)
    heatmap_optional(analysis_guidedbackprop[0], r=rate, alp=alpha)
    fig.savefig(imagefolder + str(os.path.split(sfile)[1]) + "_r=" + str(rate) + ".png", transparent=transp)
    plt.cla()
    fig.clf()
    plt.clf()
    plt.close('all')
    plt.close(fig)
    del input_img, input_img_1, analysis_guidedbackprop
    gc.collect()

    ### raw data ###
    datafile = open(rawfolder + str(os.path.split(sfile)[1]), "w")
    print(str(analysis_guidedbackprop), file=datafile)